**Image Segmentation**

In [ ]:
# Import all libraries
import os
import numpy as np
import nibabel as nib
import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import keras.backend as K
from niwidgets import NiftiWidget

# Define the dataset path
folder_path = '/content/drive/My Drive/Data'
files = os.listdir(folder_path)

In [ ]:
# List all files in the folder
file_names = os.listdir(folder_path)

# Split data into training and testing sets
train_files, test_files = train_test_split(file_names, test_size=0.1, random_state=42)

print(len(train_files))
print(len(test_files))

**Pre-processing of 3D images**

In [ ]:
# Min-max scaling
def normalizeImageIntensityRange(img):
  min_val = np.min(img)
  max_val = np.max(img)
  normalized_data = (img - min_val) / (max_val - min_val)
  return normalized_data

In [ ]:
# Read image or mask volume
def readImageVolume(imgPath, normalize=False):
    img = nib.load(imgPath).get_fdata()
    return normalizeImageIntensityRange(img)

In [ ]:
# Save volume slice to file
def saveSlice(img, fname, path):
    img = np.uint8(img * 255)
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')

In [ ]:
SLICE_X = True
SLICE_Y = True
SLICE_Z = False

SLICE_DECIMATE_IDENTIFIER = 3

# Slice image in all directions and save
def sliceAndSaveVolumeImage(vol, fname, path):
    (dimx, dimy, dimz) = vol.shape
    print(dimx, dimy, dimz)
    cnt = 0
    if SLICE_X:
        cnt += dimx
        print('Slicing X: ')
        for i in range(dimx):
            saveSlice(vol[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', path)

    if SLICE_Y:
        cnt += dimy
        print('Slicing Y: ')
        for i in range(dimy):
            saveSlice(vol[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', path)

    if SLICE_Z:
        cnt += dimz
        print('Slicing Z: ')
        for i in range(dimz):
            saveSlice(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path)
    return cnt

In [ ]:
# Read and process image volumes
def imageVolumes(files):
  for folder in files:
    path = os.path.join(folder_path, folder)
    for file in sorted(os.listdir(path)):
      if "ROI" not in file:
        img = readImageVolume(os.path.join(path, file), False)
        print(file, img.shape, np.sum(img.shape), np.min(img), np.max(img))
        numOfSlices = sliceAndSaveVolumeImage(img, os.path.basename(path), imageOutput)
        print(f'\n{file}, {numOfSlices} slices created \n')

In [ ]:
# Read and process image mask volumes
def imagemaskVolumes(files):
  for folder in files:
    path = os.path.join(folder_path, folder)
    for file in sorted(os.listdir(path)):
      if "ROI" in file:
        img = readImageVolume(os.path.join(path, file), False)
        print(file, img.shape, np.sum(img.shape), np.min(img), np.max(img))
        numOfSlices = sliceAndSaveVolumeImage(img, os.path.basename(path), imagemaskOutput)
        print(f'\n{file}, {numOfSlices} slices created \n')

In [ ]:
# Create a file for saving the image and mask for train
imageOutput = '/content/train/img'
os.makedirs(imageOutput, exist_ok=True)
imagemaskOutput = '/content/train/mask'
os.makedirs(imagemaskOutput, exist_ok=True)

# Process image and  mask for trianing files
imageVolumes(train_files)
imagemaskVolumes(train_files)

In [ ]:
# Create a file for saving the image and mask for validation
imageOutput = '/content/test/img'
os.makedirs(imageOutput, exist_ok=True)
imagemaskOutput = '/content/test/mask'
os.makedirs(imagemaskOutput, exist_ok=True)

# Process image and  mask for testing files
imageVolumes(test_files)
imagemaskVolumes(test_files)

In [ ]:
 # Define constants
SEED = 909
BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST = 16

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

data_dir_train_image = 'train/img'
data_dir_train_mask = 'train/mask'
data_dir_test_image = 'test/img'
data_dir_test_mask = 'test/mask'

NUM_TRAIN = 16384
NUM_TEST = 5120

NUM_OF_EPOCHS = 30

In [ ]:
# Data agumentation for training images and mask
def create_segmentation_generator_train(img_path, msk_path, BATCH_SIZE):
    img_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.3)

    img_generator = img_datagen.flow_from_directory(
        img_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        seed=SEED)
    msk_generator = img_datagen.flow_from_directory(
        msk_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        seed=SEED)

    return zip(img_generator, msk_generator)

In [ ]:
# Data agumentation for testing images and mask
def create_segmentation_generator_test(img_path, msk_path, BATCH_SIZE):
    img_datagen = ImageDataGenerator(rescale=1./255)

    img_generator = img_datagen.flow_from_directory(
        img_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        seed=SEED)
    msk_generator = img_datagen.flow_from_directory(
        msk_path,
        target_size=IMG_SIZE,
        class_mode=None,
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        seed=SEED)

    return zip(img_generator, msk_generator)

In [ ]:
# Call the traing images
train_generator = create_segmentation_generator_train(
    data_dir_train_image,
    data_dir_train_mask,
    BATCH_SIZE_TRAIN)

# Call the testing images
test_generator = create_segmentation_generator_test(
    data_dir_test_image,
    data_dir_test_mask,
    BATCH_SIZE_TEST)

In [ ]:
# Display image after agumentation
def display(display_list):
    plt.figure(figsize=(15,15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    plt.show()

In [ ]:
# Display image, True mask and Predicted mask from test_generator
def show_dataset(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        display([image[0], mask[0]])

In [ ]:
show_dataset(train_generator, 2)

**Training the dataset**

In [ ]:
# Defining Unet model from scratch
def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=1):
    inputs = keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, in_channels))
    x = inputs

    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same', kernel_initializer='he_normal')

    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
            if _ == 0:
                x = keras.layers.Dropout(0.25)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.Normalization()(x)
            x = keras.layers.MaxPool2D(pooling_size)(x)

    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv2DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Normalization()(x)
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)

    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv2D(out_channels, kernel_size=1, activation=activation, padding='same')(x)

    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')

# Defining dice loss for image segmantation
def dice_loss(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    dice_loss = 1.0 - (2.0 * intersection + K.epsilon()) / (union + K.epsilon())
    return dice_loss

In [ ]:
# define constants
EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST

model = unet(4)
model.compile(optimizer='adam', loss=dice_loss, metrics=['accuracy'])

In [ ]:
# Summary of overall model
model.summary()

In [ ]:
# Train the model
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=EPOCH_STEP_TRAIN,
    validation_data=test_generator,
    validation_steps=EPOCH_STEP_TEST,
    epochs=NUM_OF_EPOCHS)

In [ ]:
# Save the model as an HDF5 file
model.save(f'UNET-ToothSegmentation_{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5')

In [ ]:
def show_prediction(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        pred_mask = model.predict(image)[0] > 0.5
        display([image[0], mask[0], pred_mask])

In [ ]:
show_prediction(test_generator, 3)

In [ ]:
# Define constants
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

# For single slicing prediction
sliceIndex = 24

In [ ]:
# Min-max scaling
def normalizeImageIntensityRange(img):
  min_val = np.min(img)
  max_val = np.max(img)
  normalized_data = (img - min_val) / (max_val - min_val)
  return normalized_data

In [ ]:
# Pick any image from test to predict the mask
test

In [ ]:
# Pick any image
targetName = 'N002'
targetImagePath = f'/content/drive/My Drive/Data/{targetName}/{targetName}.nii.gz'
targetMaskPath  = f'/content/drive/My Drive/Data/{targetName}/{targetName}_ROI.nii.gz'

imgTargetNii = nib.load(targetImagePath)
imgMaskNii = nib.load(targetMaskPath)

imgTarget = normalizeImageIntensityRange(imgTargetNii.get_fdata())
imgMask = imgMaskNii.get_fdata()

In [ ]:
# Load the model
model = load_model(f'UNET-ToothSegmentation_{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5')

In [ ]:
def scaleImg(img, height, width):
    return cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_LINEAR)

**Single Slicing Prediction**

In [ ]:
# show input image slice
plt.figure(figsize=(15,15))
imgSlice = imgTarget[sliceIndex,:,:]
imgDimX, imgDimY = imgSlice.shape
imgSliceScaled = scaleImg(imgSlice, IMAGE_HEIGHT, IMAGE_WIDTH)
plt.subplot(1,2,1)
plt.imshow(imgSlice, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(imgSliceScaled, cmap='gray')
plt.show()
imgSlice.shape, imgSliceScaled.shape

In [ ]:
# show input mask slice
plt.figure(figsize=(15,15))
maskSlice = imgMask[sliceIndex,:,:]
maskSliceScaled = scaleImg(maskSlice, IMAGE_HEIGHT, IMAGE_WIDTH)
plt.subplot(1,2,1)
plt.imshow(maskSlice, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(maskSliceScaled, cmap='gray')
plt.show()
maskSlice.shape, maskSliceScaled.shape

In [ ]:
# Predict with UNET model
plt.figure(figsize=(15,15))
imageInput = imgSliceScaled[np.newaxis,:,:,np.newaxis]
maskPredict = model.predict(imageInput)[0,:,:,0]
maskPredictScaled = scaleImg(maskPredict, imgDimX, imgDimY)
plt.subplot(1,2,2)
plt.imshow(maskPredict, cmap='gray')
plt.subplot(1,2,1)
plt.imshow(maskPredictScaled, cmap='gray')
plt.show()
maskPredictScaled.shape, maskPredict.shape

**Prediction of full volume**

In [ ]:
SLICE_X = True
SLICE_Y = True
SLICE_Z = False

# Slice image in all directions
def predictVolume(inImg, toBin=True):
    (xMax, yMax, zMax) = inImg.shape

    outImgX = np.zeros((xMax, yMax, zMax))
    outImgY = np.zeros((xMax, yMax, zMax))
    outImgZ = np.zeros((xMax, yMax, zMax))

    cnt = 0.0
    if SLICE_X:
        cnt += 1.0
        for i in range(xMax):
            img = scaleImg(inImg[i,:,:], IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
            tmp = model.predict(img)[0,:,:,0]
            outImgX[i,:,:] = scaleImg(tmp, yMax, zMax)
    if SLICE_Y:
        cnt += 1.0
        for i in range(yMax):
            img = scaleImg(inImg[:,i,:], IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
            tmp = model.predict(img)[0,:,:,0]
            outImgY[:,i,:] = scaleImg(tmp, xMax, zMax)
    if SLICE_Z:
        cnt += 1.0
        for i in range(zMax):
            img = scaleImg(inImg[:,:,i], IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
            tmp = model.predict(img)[0,:,:,0]
            outImgZ[:,:,i] = scaleImg(tmp, xMax, yMax)

    outImg = (outImgX + outImgY + outImgZ)/cnt
    if(toBin):
        outImg[outImg>0.5] = 1.0
        outImg[outImg<=0.5] = 0.0
    return outImg

In [ ]:
predImg = predictVolume(imgTarget)

In [ ]:
my_widget = NiftiWidget(imgTargetNii)
my_widget.nifti_plotter(colormap='gray')

In [ ]:
my_widget = NiftiWidget(nib.dataobj_images.DataobjImage(predImg))
my_widget.nifti_plotter(colormap='gray')

In [ ]:
my_widget = NiftiWidget(imgMaskNii)
my_widget.nifti_plotter(colormap='gray')